In [8]:
import numpy as np
import pandas as pd


# Load the data
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
# sample_submission = pd.read_csv("sample_submission.csv")
# sample_submission_y = sample_submission.drop(columns=["UID"])

# Extract features and target
X_train = train_data.iloc[500:, 1:-1]
y_train = train_data.iloc[500:, -1]
X_val = train_data.iloc[:500, 1:-1]
y_val = train_data.iloc[:500, -1]
X_test = test_data.iloc[:, 1:]

# Categorical columns
categorical_cols = ['col_0', 'col_1', 'col_2', 'col_3', 'col_5', 'col_6']

# # Fill NaN values in categorical columns with 'Unknown'
# def fill_na(data, categorical_cols):
#     for col in categorical_cols:
#         data[col].fillna('Unknown', inplace=True)
#     return data

# Fill NaN values in categorical columns with the most frequent value in the respective columns
def fill_na(data, categorical_cols):
    for col in categorical_cols:
        most_frequent_value = data[col].mode().iloc[0]  # Get the mode (most frequent value)
        data[col].fillna(most_frequent_value, inplace=True)
    return data

X_train = fill_na(X_train, categorical_cols)
X_val = fill_na(X_val, categorical_cols)
X_test = fill_na(X_test, categorical_cols)

# label encoding
def label_encoding(data, categorical_cols):
    label_encoders = {}
    for col in categorical_cols:
        unique_categories = data[col].unique()
        encoding_map = {category: index + 1 for index, category in enumerate(unique_categories)}
        data[col] = data[col].map(encoding_map)
        label_encoders[col] = encoding_map
    return data

X_train = label_encoding(X_train, categorical_cols)
X_val = label_encoding(X_val, categorical_cols)
X_test = label_encoding(X_test, categorical_cols)

# normalize
def normalize(df):
    return (df - df.min()) / (df.max() - df.min())

X_train = normalize(X_train)
X_val = normalize(X_val)
X_test = normalize(X_test)

# convert df into numpy
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_val = X_val.to_numpy()
y_val = y_val.to_numpy()
X_test = X_test.to_numpy()
# sample_submission_y = sample_submission_y.to_numpy()

In [9]:
from collections import Counter

def entropy(y):
    hist = np.bincount(y)
    ps = hist / len(y)
    return -np.sum([p * np.log2(p) for p in ps if p > 0])


class Node:
    def __init__(
        self, feature=None, threshold=None, left=None, right=None, *, value=None
    ):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None


class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_feats=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.root = None

    def fit(self, X, y):
        self.n_feats = X.shape[1] if not self.n_feats else min(self.n_feats, X.shape[1])
        self.root = self._grow_tree(X, y)

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))

        # stopping criteria
        if (
            depth >= self.max_depth
            or n_labels == 1
            or n_samples < self.min_samples_split
        ):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_features, self.n_feats, replace=False)

        # greedily select the best split according to information gain
        best_feat, best_thresh = self._best_criteria(X, y, feat_idxs)

        # grow the children that result from the split
        left_idxs, right_idxs = self._split(X[:, best_feat], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1)
        return Node(best_feat, best_thresh, left, right)

    def _best_criteria(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_thresh = None, None
        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)
            for threshold in thresholds:
                gain = self._information_gain(y, X_column, threshold)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_thresh = threshold

        return split_idx, split_thresh

    def _information_gain(self, y, X_column, split_thresh):
        # parent loss
        parent_entropy = entropy(y)

        # generate split
        left_idxs, right_idxs = self._split(X_column, split_thresh)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        # compute the weighted avg. of the loss for the children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = entropy(y[left_idxs]), entropy(y[right_idxs])
        child_entropy = (n_l / n) * e_l + (n_r / n) * e_r

        # information gain is difference in loss before vs. after split
        ig = parent_entropy - child_entropy
        return ig

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

    def _most_common_label(self, y):
        counter = Counter(y)
        most_common = counter.most_common(1)[0][0]
        return most_common

In [10]:
def bootstrap_sample(X, y):
    n_samples = X.shape[0]
    idxs = np.random.choice(n_samples, n_samples, replace=True)
    return X[idxs], y[idxs]


def most_common_label(y):
    counter = Counter(y)
    most_common = counter.most_common(1)[0][0]
    return most_common

In [12]:
def calculate_rmse(y_true, y_pred):
    squared_errors = (y_true - y_pred) ** 2
    mean_squared_error = np.mean(squared_errors)
    rmse = np.sqrt(mean_squared_error)
    return rmse

In [13]:
class RandomForestRegressor:
    def __init__(self, n_trees=10, min_samples_split=2, max_depth=100, n_feats=None):
        self.n_trees = n_trees
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTree(
                min_samples_split=self.min_samples_split,
                max_depth=self.max_depth,
                n_feats=self.n_feats,
            )
            X_samp, y_samp = bootstrap_sample(X, y)
            tree.fit(X_samp, y_samp)
            self.trees.append(tree)

    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        y_pred = np.mean(tree_preds, axis=0)
        return y_pred

# Your bootstrap_sample and most_common_label functions remain the same...

# Create a RandomForestRegressor instance
rf_regressor = RandomForestRegressor(n_trees=100, min_samples_split=3, max_depth=3)

# Fit the RandomForestRegressor on your data
rf_regressor.fit(X_train, y_train)

# Predict on the validation set
val_pred_rf = rf_regressor.predict(X_val)

# Calculate RMSE on the validation set
val_rmse_rf = calculate_rmse(val_pred_rf, y_val)
print("Validation RMSE for Random Forest Regression:", val_rmse_rf)

# Predict on the test set
test_pred_rf = rf_regressor.predict(X_test)

# Save predictions to a CSV file
result_df_rf = pd.DataFrame({'UID': test_data['UID'], 'y': test_pred_rf})
result_df_rf.to_csv('RandomForestRegression.csv', index=False)

Validation RMSE for Random Forest Regression: 57496.01740002939
